In [1]:
import scipy
scipy.__version__

'1.4.1'

In [2]:
import numpy
import os
import os.path
import joblib
from Datasets import *
from Settings import *
from Estimators import *
from Metrics import *
from Policy import *
import scipy.stats
import numpy as np

In [3]:
for foldID in range(1,2):
    for fraction in ['train']:
        mslrData=Datasets()
        mslrData.loadTxt(Settings.DATA_DIR+'MSLR-WEB10K/Fold'+str(foldID)+'/'+fraction+'.txt', 'MSLR10k-'+str(foldID)+'-'+fraction)

...........................................................
Datasets:loadTxt [INFO] Loaded /Users/haohongxu/slates_semisynth_expts/MSLR-WEB10K/Fold1/train.txt 	 NumQueries 6000 	 [Min/Max]DocsPerQuery 1 809


In [4]:
data=mslrData

In [5]:
data

In [7]:
anchorURLFeatures, bodyTitleDocFeatures=Settings.get_feature_sets("MSLR10k")

class Policy:
    #dataset: (Datasets) Must be initialized using Datasets.loadTxt(...)/loadNpz(...)
    #allow_repetitions: (bool) If true, the policy predicts rankings with repeated documents
    def __init__(self, dataset, allow_repetitions):
        self.dataset=dataset
        self.allowRepetitions=allow_repetitions
        self.name=None
        ###All sub-classes of Policy should supply a predict method
        ###Requires: (int) query_id; (int) ranking_size.
        ###Returns: list[int],length=min(ranking_size,docsPerQuery[query_id]) ranking
class DeterministicPolicy(Policy):
    #model_type: (str) Model class to use for scoring documents
    def __init__(self, dataset, model_type, regress_gains=False, weighted_ls=False, hyper_params=None):
        Policy.__init__(self, dataset, False)
        self.modelType=model_type
        self.hyperParams={'alpha': (numpy.logspace(-3,2,num=6,base=10)).tolist()}
        if hyper_params is not None:
            self.hyperParams=hyper_params
        
        self.regressGains=regress_gains
        self.weighted=weighted_ls
        
        self.treeDepths={'max_depth': list(range(3,21,3))}
        
        #Must call train(...) to set all these members
        #before using DeterministicPolicy objects elsewhere
        self.featureList=None
        if self.modelType=='tree':
            self.tree=None
        else:
            self.policyParams=None
            
        #These members are set by predictAll(...) method
        self.savedRankingsSize=None
        self.savedRankings=None
        
        print("DeterministicPolicy:init [INFO] Dataset", dataset.name, flush=True)
    
    #feature_list: list[int],length=unmaskedFeatures; List of features that should be used for training
    #name: (str) String to help identify this DeterministicPolicy object henceforth
    def train(self, feature_list, name):
        self.featureList=feature_list
        self.name=name+'-'+self.modelType
        modelFile=Settings.DATA_DIR+self.dataset.name+'_'+self.name
        if 'alpha' not in self.hyperParams:
            #Expecting hyper-params for GBRT; Add those hyper-params to the model file name
            modelFile=modelFile+'ensemble-'+str(self.hyperParams['ensemble'])+'_lr-'+str(self.hyperParams['lr'])+'_subsample-'+str(self.hyperParams['subsample'])+'_leaves-'+str(self.hyperParams['leaves'])
            
        if self.modelType=='tree' or self.modelType=='gbrt':
            modelFile+='.z'
        else:
            modelFile+='.npz'
            
        self.savedRankingsSize=None
        self.savedRankings=None
        
        if os.path.exists(modelFile):
            if self.modelType=='tree' or self.modelType=='gbrt':
                self.tree=joblib.load(modelFile)
                print("DeterministicPolicy:train [INFO] Using precomputed policy", modelFile, flush=True)
            else:
                with numpy.load(modelFile) as npFile:
                    self.policyParams=npFile['policyParams']
                print("DeterministicPolicy:train [INFO] Using precomputed policy", modelFile, flush=True)
                print("DeterministicPolicy:train [INFO] PolicyParams", self.policyParams,flush=True)
        else:
            numQueries=len(self.dataset.features)
        
            allFeatures=None
            allTargets=None
            print("DeterministicPolicy:train [INFO] Constructing features and targets", flush=True)
                
            if self.dataset.mask is None:
                allFeatures=scipy.sparse.vstack(self.dataset.features, format='csc')
                allTargets=numpy.hstack(self.dataset.relevances)
            else:
                temporaryFeatures=[]
                temporaryTargets=[]
                for currentQuery in range(numQueries):
                    temporaryFeatures.append(self.dataset.features[currentQuery][self.dataset.mask[currentQuery], :])
                    temporaryTargets.append(self.dataset.relevances[currentQuery][self.dataset.mask[currentQuery]])
                
                allFeatures=scipy.sparse.vstack(temporaryFeatures, format='csc')
                allTargets=numpy.hstack(temporaryTargets)
        
            if self.regressGains:
                allTargets=numpy.exp2(allTargets)-1.0
            
            allSampleWeights=None
            fitParams=None
            if self.weighted:
                allSampleWeights=numpy.array(self.dataset.docsPerQuery, dtype=numpy.float64)
                allSampleWeights=numpy.reciprocal(allSampleWeights)
                allSampleWeights=numpy.repeat(allSampleWeights, self.dataset.docsPerQuery)    
                fitParams={'sample_weight': allSampleWeights}
            
            #Restrict features to only the unmasked features
            if self.featureList is not None:
                print("DeterministicPolicy:train [INFO] Masking unused features. Remaining feature size", 
                    len(feature_list), flush=True)
                allFeatures = allFeatures[:, self.featureList]
        
            print("DeterministicPolicy:train [INFO] Beginning training", self.modelType, flush=True)
            if self.modelType=='tree':
                treeCV=sklearn.model_selection.GridSearchCV(sklearn.tree.DecisionTreeRegressor(criterion="mse",
                                                        splitter="random", min_samples_split=4, 
                                                        min_samples_leaf=4),
                                param_grid=self.treeDepths,
                                scoring=None, n_jobs=-2,
                                cv=5, refit=True, verbose=0, pre_dispatch="1*n_jobs",
                                error_score='raise', return_train_score=False)
                            
                treeCV.fit(allFeatures, allTargets)
                self.tree=treeCV.best_estimator_
                print("DeterministicPolicy:train [INFO] Done. Best depth", 
                            treeCV.best_params_['max_depth'], flush=True)
                joblib.dump(self.tree, modelFile, compress=9, protocol=-1)
            
            elif self.modelType=='lasso':
                lassoCV=sklearn.model_selection.GridSearchCV(sklearn.linear_model.Lasso(fit_intercept=False,
                                                        normalize=False, precompute=False, copy_X=False, 
                                                        max_iter=3000, tol=1e-4, warm_start=False, positive=False,
                                                        random_state=None, selection='random'),
                                param_grid=self.hyperParams,
                                scoring=None, n_jobs=-2,
                                cv=5, refit=True, verbose=0, pre_dispatch="1*n_jobs",
                                error_score='raise', return_train_score=False)
                                
                lassoCV.fit(allFeatures, allTargets)
                self.policyParams=lassoCV.best_estimator_.coef_
                print("DeterministicPolicy:train [INFO] Done. CVAlpha", lassoCV.best_params_['alpha'], flush=True)
                print("DeterministicPolicy:train [INFO] PolicyParams", self.policyParams,flush=True)
                numpy.savez_compressed(modelFile, policyParams=self.policyParams)
        
            elif self.modelType == 'ridge':
                ridgeCV=sklearn.model_selection.GridSearchCV(sklearn.linear_model.Ridge(fit_intercept=False,
                                                                                    normalize=False, copy_X=False,
                                                                                    max_iter=3000, tol=1e-4, random_state=None),
                                                         param_grid=self.hyperParams,
                                                         n_jobs=-2, 
                                                         cv=3, refit=True, verbose=0, pre_dispatch='1*n_jobs')
                ridgeCV.fit(allFeatures, allTargets)
                self.policyParams=ridgeCV.best_estimator_.coef_
                print("DeterministicPolicy:train [INFO] Done. CVAlpha", ridgeCV.best_params_['alpha'], flush=True)
            elif self.modelType=='gbrt':
                tree=sklearn.ensemble.GradientBoostingRegressor(learning_rate=self.hyperParams['lr'],
                            n_estimators=self.hyperParams['ensemble'], subsample=self.hyperParams['subsample'], max_leaf_nodes=self.hyperParams['leaves'], 
                            max_features=1.0)
                tree.fit(allFeatures, allTargets, sample_weight=allSampleWeights)
                self.tree=tree
                print("DeterministicPolicy:train [INFO] Done.", flush=True)
                joblib.dump(self.tree, modelFile, compress=9, protocol=-1)
            
            else:
                print("DeterministicPolicy:train [ERR] %s not supported." % self.modelType, flush=True)
                sys.exit(0)
    
    #query_id: (int) Query ID in self.dataset
    #ranking_size: (int) Size of ranking. Returned ranking length is min(ranking_size,docsPerQuery[query_id])
    #                       Use ranking_size=-1 to rank all available documents for query_id
    def predict(self, query_id, ranking_size):
        if self.savedRankingsSize is not None and self.savedRankingsSize==ranking_size:
            return self.savedRankings[query_id]
        
        allowedDocs=self.dataset.docsPerQuery[query_id]
        validDocs=ranking_size
        if ranking_size <= 0 or validDocs > allowedDocs:
            validDocs=allowedDocs
        
        currentFeatures=None
        if self.dataset.mask is None:
            if self.featureList is not None:
                currentFeatures=self.dataset.features[query_id][:, self.featureList]
            else:
                currentFeatures=self.dataset.features[query_id]
            
        else:
            currentFeatures=self.dataset.features[query_id][self.dataset.mask[query_id], :]
            if self.featureList is not None:
                currentFeatures=currentFeatures[:, self.featureList]
        
        allDocScores=None
        if self.modelType=='tree':
            allDocScores=self.tree.predict(currentFeatures)
        elif self.modelType=='gbrt':
            allDocScores=self.tree.predict(currentFeatures.toarray())
        else:
            allDocScores=currentFeatures.dot(self.policyParams)
            
        tieBreaker=numpy.random.random(allDocScores.size)
        sortedDocScores=numpy.lexsort((tieBreaker,-allDocScores))[0:validDocs]
        if self.dataset.mask is None:
            return sortedDocScores
        else:
            return self.dataset.mask[query_id][sortedDocScores]
    
    #ranking_size: (int) Size of ranking. Returned ranking length is min(ranking_size,docsPerQuery[query_id])
    #                       Use ranking_size=-1 to rank all available documents for query_id
    def predictAll(self, ranking_size):
        if self.savedRankingsSize is not None and self.savedRankingsSize==ranking_size:
            return
            
        numQueries=len(self.dataset.features)
        predictedRankings=[]
        for i in range(numQueries):
            predictedRankings.append(self.predict(i, ranking_size))
                
            if i%100==0:
                print(".", end="", flush=True)
                
        self.savedRankingsSize=ranking_size
        self.savedRankings=predictedRankings
        print("", flush=True)
        print("DeterministicPolicy:predictAll [INFO] Generated all predictions for %s using policy: " %
                self.dataset.name, self.name, flush=True)
        
    #num_allowed_docs: (int) Filters the dataset where the max docs per query is num_allowed_docs.
    #                        Uses policyParams to rank and filter the original document set.
    def filterDataset(self, num_allowed_docs):
        self.savedRankingsSize=None
        self.savedRankings=None
        
        numQueries=len(self.dataset.docsPerQuery)
        
        self.dataset.name=self.dataset.name+'-filt('+self.name+'-'+str(num_allowed_docs)+')'
        
        newMask = []
        for i in range(numQueries):
            producedRanking=self.predict(i, num_allowed_docs)
            self.dataset.docsPerQuery[i]=numpy.shape(producedRanking)[0]
            newMask.append(producedRanking)
            if i%100==0:
                print(".", end="", flush=True)
                
        self.dataset.mask=newMask
        print("", flush=True)
        print("DeterministicPolicy:filteredDataset [INFO] New Name", self.dataset.name, "\t MaxNumDocs", num_allowed_docs, flush=True)

class UniformPolicy(Policy):
    def __init__(self, dataset, allow_repetitions):
        Policy.__init__(self, dataset, allow_repetitions)
        self.name='Unif-'
        if allow_repetitions:
            self.name+='Rep'
        else:
            self.name+='NoRep'
    
        #These members are set on-demand by setupGamma(...)
        self.gammas=None
        self.gammaRankingSize=None
        
        print("UniformPolicy:init [INFO] Dataset: %s AllowRepetitions:" % dataset.name,
                        allow_repetitions, flush=True)
    
    #ranking_size: (int) Size of ranking.
    def setupGamma(self, ranking_size):
        if self.gammaRankingSize is not None and self.gammaRankingSize==ranking_size:
            print("UniformPolicy:setupGamma [INFO] Gamma has been pre-computed for this ranking_size. Size of Gamma cache:", len(self.gammas), flush=True)
            return
        
        gammaFile=Settings.DATA_DIR+self.dataset.name+'_'+self.name+'_'+str(ranking_size)+'.z'
        if os.path.exists(gammaFile):
            self.gammas=joblib.load(gammaFile)
            self.gammaRankingSize=ranking_size
            print("UniformPolicy:setupGamma [INFO] Using precomputed gamma", gammaFile, flush=True)
            
        else:
            self.gammas={}
            self.gammaRankingSize=ranking_size
            
            candidateSet=set(self.dataset.docsPerQuery)
            
            responses=joblib.Parallel(n_jobs=-2, verbose=50)(joblib.delayed(UniformGamma)(i, ranking_size, self.allowRepetitions) for i in candidateSet)
            
            for tup in responses:
                self.gammas[tup[0]]=tup[1]
            
            joblib.dump(self.gammas, gammaFile, compress=9, protocol=-1)
            print("", flush=True)
            print("UniformPolicy:setupGamma [INFO] Finished creating Gamma_pinv cache. Size", len(self.gammas), flush=True)

    def predict(self, query_id, ranking_size):
        allowedDocs=self.dataset.docsPerQuery[query_id]    
        
        validDocs=ranking_size
        if ranking_size < 0 or ((not self.allowRepetitions) and (validDocs > allowedDocs)):
            validDocs=allowedDocs
            
        producedRanking=None
        if self.allowRepetitions:
            producedRanking=numpy.random.choice(allowedDocs, size=validDocs,
                                replace=True)
        else:
            producedRanking=numpy.random.choice(allowedDocs, size=validDocs,
                                replace=False)
                                
        if self.dataset.mask is None:
            return producedRanking
        else:
            return self.dataset.mask[query_id][producedRanking]
        

class NonUniformPolicy(Policy):
    def __init__(self, deterministic_policy, dataset, allow_repetitions, decay):
        Policy.__init__(self, dataset, allow_repetitions)
        self.decay = decay
        self.policy = deterministic_policy
        self.name='NonUnif-'
        if allow_repetitions:
            self.name+='Rep'
        else:
            self.name+='NoRep'
        self.name += '(' + deterministic_policy.name + ';' + str(decay) + ')'
        
        #These members are set on-demand by setupGamma
        self.gammas=None
        self.multinomials=None
        self.gammaRankingSize=None
        
        print("NonUniformPolicy:init [INFO] Dataset: %s AllowRepetitions:" % dataset.name,
                        allow_repetitions, "\t Decay:", decay, flush=True)
    
    
    def setupGamma(self, ranking_size):
        if self.gammaRankingSize is not None and self.gammaRankingSize==ranking_size:
            print("NonUniformPolicy:setupGamma [INFO] Gamma has been pre-computed for this ranking_size. Size of Gamma cache:", len(self.gammas), flush=True)
            return
        
        gammaFile=Settings.DATA_DIR+self.dataset.name+'_'+self.name+'_'+str(ranking_size)+'.z'
        if os.path.exists(gammaFile):
            self.gammas, self.multinomials=joblib.load(gammaFile)
            self.gammaRankingSize=ranking_size
            print("NonUniformPolicy:setupGamma [INFO] Using precomputed gamma", gammaFile, flush=True)
            
        else:
            self.gammas={}
            self.multinomials={}
            self.gammaRankingSize=ranking_size
            
            candidateSet=set(self.dataset.docsPerQuery)
            responses=joblib.Parallel(n_jobs=-2, verbose=50)(joblib.delayed(NonUniformGamma)(i, self.decay, ranking_size, self.allowRepetitions) for i in candidateSet)
            
            for tup in responses:
                self.gammas[tup[0]]=tup[2]
                self.multinomials[tup[0]]=tup[1]
            
            joblib.dump((self.gammas, self.multinomials), gammaFile, compress=9, protocol=-1)
            print("", flush=True)
            print("NonUniformPolicy:setupGamma [INFO] Finished creating Gamma_pinv cache. Size", len(self.gammas), flush=True)

        self.policy.predictAll(-1)

    def predict(self, query_id, ranking_size):
        allowedDocs=self.dataset.docsPerQuery[query_id]    
        underlyingRanking=self.policy.predict(query_id, -1)
            
        validDocs=ranking_size
        if ranking_size < 0 or ((not self.allowRepetitions) and (validDocs > allowedDocs)):
            validDocs=allowedDocs
            
        currentDistribution=self.multinomials[allowedDocs]
        producedRanking=None
        if self.allowRepetitions:
            producedRanking=numpy.random.choice(allowedDocs, size=validDocs,
                                replace=True, p=currentDistribution)
        else:
            producedRanking=numpy.random.choice(allowedDocs, size=validDocs,
                                replace=False, p=currentDistribution)
                                
        return underlyingRanking[producedRanking]

In [15]:
import imp
imp.reload(Policy)
numpy.random.seed(args.numpy_seed)
detLogger=Policy.DeterministicPolicy(data, 'tree')
detLogger.train(anchorURLFeatures, 'url')

detLogger.filterDataset(args.max_docs)
data=detLogger.dataset
del detLogger

DeterministicPolicy:init [INFO] Dataset MSLR10k-1-train
DeterministicPolicy:train [INFO] Constructing features and targets
DeterministicPolicy:train [INFO] Masking unused features. Remaining feature size 57
DeterministicPolicy:train [INFO] Beginning training tree
DeterministicPolicy:train [INFO] Done. Best depth 9
............................................................
DeterministicPolicy:filteredDataset [INFO] New Name MSLR10k-1-train-filt(url-tree-100) 	 MaxNumDocs 100


In [16]:
import Datasets
import argparse
import Settings
import sys
import os
import numpy
import Policy
import Metrics
import Estimators
import joblib
parser = argparse.ArgumentParser(description='Synthetic Testbed Experiments.')

In [25]:
parser= argparse.ArgumentParser(description='Synthetic Testbed Experiments.')

In [26]:
parser.add_argument('--max_docs', '-m', metavar='M', type=int, help='Filter documents',
                        default=100)
parser.add_argument('--length_ranking', '-l', metavar='L', type=int, help='Ranking Size',
                    default=10)
parser.add_argument('--replacement', '-r', metavar='R', type=bool, help='Sampling with or without replacement',
                    default=False)
parser.add_argument('--temperature', '-t', metavar='T', type=float, help='Temperature for logging policy', 
                    default=1.0)        #Use 0 < temperature < 2 to have reasonable tails for logger [-t 2 => smallest prob is 10^-4 (Uniform is 10^-2)]
parser.add_argument('--logging_ranker', '-f', metavar='F', type=str, help='Model for logging ranker', 
                    default="tree", choices=["tree", "lasso"])
parser.add_argument('--evaluation_ranker', '-e', metavar='E', type=str, help='Model for evaluation ranker', 
                    default="lasso", choices=["tree", "lasso"])
parser.add_argument('--dataset', '-d', metavar='D', type=str, help='Which dataset to use',
                    default="MSLR10k", choices=["MSLR", "MSLR10k", "MQ2008", "MQ2007"])
parser.add_argument('--value_metric', '-v', metavar='V', type=str, help='Which metric to evaluate',
                    default="ERR", choices=["NDCG", "ERR", "MaxRel", "SumRel"])
parser.add_argument('--numpy_seed', '-n', metavar='N', type=int, 
                    help='Seed for numpy.random', default=387)
parser.add_argument('--output_dir', '-o', metavar='O', type=str, 
                    help='Directory to store pkls', default=Settings.DATA_DIR)
parser.add_argument('--approach', '-a', metavar='A', type=str, 
                    help='Approach name', default='IPS', choices=["OnPolicy", "IPS", "IPS_SN", "PI", "PI_SN", "DM_tree", "DM_lasso", "DMc_lasso", "DM_ridge", "DMc_ridge"])
parser.add_argument('--logSize', '-s', metavar='S', type=int, 
                    help='Size of log', default=30000)
parser.add_argument('--trainingSize', '-z', metavar='Z', type=int, 
                    help='Size of training data for direct estimators', default=100)
parser.add_argument('--saveSize', '-u', metavar='U', type=int, 
                    help='Number of saved datapoints', default=10000)
parser.add_argument('--start', type=int,
                    help='Starting iteration number', default=1)
parser.add_argument('--stop', type=int,
                    help='Stopping iteration number', default=10)

_StoreAction(option_strings=['--stop'], dest='stop', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='Stopping iteration number', metavar=None)

In [27]:
args = parser.parse_args(args=[])

In [28]:
data

In [73]:
##import imp
##imp.reload(Policy)
#Setup target policy
numpy.random.seed(args.numpy_seed)
##targetPolicy=DeterministicPolicy(data, 'tree')
##targetPolicy.train(bodyTitleDocFeatures, 'body')
##targetPolicy.predictAll(args.length_ranking)

loggingPolicy=None
if args.temperature <= 0.0:
    loggingPolicy=UniformPolicy(data, args.replacement)

else:
    underlyingPolicy=Policy.DeterministicPolicy(data, args.logging_ranker)
    underlyingPolicy.train(anchorURLFeatures, 'url')
    loggingPolicy=NonUniformPolicy(underlyingPolicy, data, args.replacement, args.temperature)
    
    
    ## traget
    underlyingPolicy2=Policy.DeterministicPolicy(data, args.logging_ranker)
    underlyingPolicy2.train(anchorURLFeatures, 'url')
    targetPolicy=NonUniformPolicy(underlyingPolicy2, data, args.replacement, args.temperature)



loggingPolicy.setupGamma(args.length_ranking)
targetPolicy.setupGamma(args.length_ranking)

smallestProb=1.0
docSet=set(data.docsPerQuery)
for i in docSet:
    currentMin=None
    if args.temperature > 0.0:
        currentMin=numpy.amin(loggingPolicy.multinomials[i])
    else:
        currentMin=1.0/i
    if currentMin < smallestProb:
        smallestProb=currentMin
print("Parallel:main [LOG] Temperature:", args.temperature, "\t Smallest marginal probability:", smallestProb, flush=True)

DeterministicPolicy:init [INFO] Dataset MSLR10k-1-train-filt(url-tree-100)
DeterministicPolicy:train [INFO] Using precomputed policy /Users/haohongxu/slates_semisynth_expts/MSLR10k-1-train-filt(url-tree-100)_url-tree.z
NonUniformPolicy:init [INFO] Dataset: MSLR10k-1-train-filt(url-tree-100) AllowRepetitions: False 	 Decay: 1.0
DeterministicPolicy:init [INFO] Dataset MSLR10k-1-train-filt(url-tree-100)
DeterministicPolicy:train [INFO] Using precomputed policy /Users/haohongxu/slates_semisynth_expts/MSLR10k-1-train-filt(url-tree-100)_url-tree.z
NonUniformPolicy:init [INFO] Dataset: MSLR10k-1-train-filt(url-tree-100) AllowRepetitions: False 	 Decay: 1.0
NonUniformPolicy:setupGamma [INFO] Using precomputed gamma /Users/haohongxu/slates_semisynth_expts/MSLR10k-1-train-filt(url-tree-100)_NonUnif-NoRep(url-tree;1.0)_10.z
............................................................
DeterministicPolicy:predictAll [INFO] Generated all predictions for MSLR10k-1-train-filt(url-tree-100) using polic

In [74]:
metric=None
if args.value_metric=="DCG":
    metric=Metrics.DCG(data, args.length_ranking)
elif args.value_metric=="NDCG":
    metric=Metrics.NDCG(data, args.length_ranking, args.replacement)
elif args.value_metric=="ERR":
    metric=Metrics.ERR(data, args.length_ranking)
elif args.value_metric=="MaxRel":
    metric=Metrics.MaxRelevance(data, args.length_ranking)
elif args.value_metric=="SumRel":
    metric=Metrics.SumRelevance(data, args.length_ranking)
else:
    print("Parallel:main [ERR] Metric %s not supported." % args.value_metric, flush=True)
    sys.exit(0)

ERR:init [INFO] RankingSize 10


In [90]:
import Estimators
import imp
imp.reload(Estimators)

<module 'Estimators' from '/Users/haohongxu/slates_semisynth_expts/Estimators.py'>

In [94]:
import Estimators
import imp
imp.reload(Estimators)
estimator=None
if args.approach=="OnPolicy":
    estimator=Estimators.OnPolicy(args.length_ranking, loggingPolicy, targetPolicy, metric)
    estimator.estimateAll()
elif args.approach=="IPS":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformIPS_s(args.length_ranking, loggingPolicy, targetPolicy)
    else:    
        estimator=Estimators.UniformIPS(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach=="IPS_SN":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformSNIPS(args.length_ranking, loggingPolicy, targetPolicy)
    else:
        estimator=Estimators.UniformSNIPS(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach=="PI":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformPI(args.length_ranking, loggingPolicy, targetPolicy)
    else:
        estimator=Estimators.UniformPI(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach=="PI_SN":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformSNPI(args.length_ranking, loggingPolicy, targetPolicy)
    else:
        estimator=Estimators.UniformSNPI(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach.startswith("DM"):
    estimatorType=args.approach.split('_',1)[1]
    estimator=Estimators.Direct(args.length_ranking, loggingPolicy, targetPolicy, estimatorType)
else:
    print("Parallel:main [ERR] Estimator %s not supported." % args.approach, flush=True)
    sys.exit(0)

In [95]:
estimator

In [96]:
args.length_ranking

10

In [97]:
targetPolicy.predict(i, args.length_ranking)

array([12, 33, 22,  1, 37,  0, 13, 29, 36, 24])

In [98]:
### IPS LOG
import imp
imp.reload(Policy)
numQueries=len(data.docsPerQuery)
trueMetric=numpy.zeros(numQueries, dtype=numpy.float64)

for i in range(numQueries):
    trueMetric[i]=metric.computeMetric(i, targetPolicy.predict(i, args.length_ranking))
    if i%100==0:
        print(".", end="", flush=True)
print("", flush=True)

target=trueMetric.mean(dtype=numpy.float64)
print("Parallel:main [LOG] *** TARGET: ", target, flush = True)
del trueMetric

............................................................
Parallel:main [LOG] *** TARGET:  0.18709950455856483


In [99]:
saveValues = numpy.linspace(start=int(args.logSize/args.saveSize), stop=args.logSize, num=args.saveSize, endpoint=True, dtype=numpy.int)
saveValues

array([    3,     6,     9, ..., 29994, 29997, 30000])

In [100]:
outputString = args.output_dir+'ssynth_'+args.value_metric+'_'+args.dataset+'_'+'4'
if args.max_docs is None:
    outputString += '-1_'
else:
    outputString += str(args.max_docs)+'_'

outputString += str(args.length_ranking) +'_'
if args.replacement:
    outputString += 'r'
else:
    outputString += 'n'
outputString += str(float(args.temperature)) + '_' 
outputString += 'f' + args.logging_ranker + '_e' + args.evaluation_ranker + '_' + str(args.numpy_seed)
outputString += '_'+args.approach
if args.approach.startswith("DM"):
    outputString += '_'+str(args.trainingSize)

In [89]:
for iteration in range(args.start, args.stop):
    iterOutputString = outputString+'_'+str(iteration)+'.z'
    if os.path.isfile(iterOutputString):
        print("Parallel:main [LOG] *** Found %s, skipping" % iterOutputString, flush=True)
        continue

    # Reset estimator
    estimator.reset()

    # reset output
    saveMSEs = numpy.zeros(args.saveSize, numpy.float64)
    savePreds = numpy.zeros(args.saveSize, numpy.float64)

    numpy.random.seed(args.numpy_seed + 7*iteration)
    currentSaveIndex=0
    currentSaveValue=saveValues[currentSaveIndex]-1

    loggedData=None
    if args.trainingSize > 0:
        loggedData=[]

    for j in range(args.logSize):
        currentQuery=numpy.random.randint(0, numQueries)
        loggedRanking=loggingPolicy.predict(currentQuery, args.length_ranking)
        loggedValue=metric.computeMetric(currentQuery, loggedRanking)

        newRanking=targetPolicy.predict(currentQuery,args.length_ranking)

        estimatedValue=None
        if (args.trainingSize > 0 and j < args.trainingSize):
            estimatedValue=0.0
            loggedData.append((currentQuery, loggedRanking, loggedValue))
        else:
            if j==args.trainingSize:
                try:
                    estimator.train(loggedData)
                    if args.approach.startswith("DMc"):
                        estimator.estimateAll(metric=metric)
                    else:
                        estimator.estimateAll()
                except AttributeError:
                    pass

            estimatedValue=estimator.estimate(currentQuery, loggedRanking, newRanking, loggedValue)/1000

        if j==currentSaveValue:
            savePreds[currentSaveIndex]=estimatedValue
            saveMSEs[currentSaveIndex]=(estimatedValue-target)**2
            currentSaveIndex+=1
            if currentSaveIndex<args.saveSize:
                currentSaveValue=saveValues[currentSaveIndex]-1

        if j%1000==0:
            print(".", end = "", flush = True)
            numpy.random.seed(args.numpy_seed + 7*iteration + j + 1)

    print("")
    print("Parallel:main [LOG] Iter:%d Truth Estimate=%0.5f" % (iteration, target), flush = True)
    print("Parallel:main [LOG] %s Estimate=%0.5f MSE=%0.3e" % (args.approach, savePreds[-1], saveMSEs[-1]), flush=True)

    joblib.dump((saveValues, saveMSEs, savePreds, target), iterOutputString)



..............................
Parallel:main [LOG] Iter:1 Truth Estimate=0.18979
Parallel:main [LOG] IPS Estimate=0.18407 MSE=3.273e-05
..............................
Parallel:main [LOG] Iter:2 Truth Estimate=0.18979
Parallel:main [LOG] IPS Estimate=0.15963 MSE=9.096e-04
..............................
Parallel:main [LOG] Iter:3 Truth Estimate=0.18979
Parallel:main [LOG] IPS Estimate=0.24354 MSE=2.889e-03
..............................
Parallel:main [LOG] Iter:4 Truth Estimate=0.18979
Parallel:main [LOG] IPS Estimate=0.26935 MSE=6.329e-03
..............................
Parallel:main [LOG] Iter:5 Truth Estimate=0.18979
Parallel:main [LOG] IPS Estimate=0.19045 MSE=4.359e-07
..............................
Parallel:main [LOG] Iter:6 Truth Estimate=0.18979
Parallel:main [LOG] IPS Estimate=0.28963 MSE=9.967e-03
..............................
Parallel:main [LOG] Iter:7 Truth Estimate=0.18979
Parallel:main [LOG] IPS Estimate=0.31394 MSE=1.541e-02
..............................
Parallel:main [LO

In [ ]:
import matplotlib.pyplot as plt
plt.plot(saveMSEs)

In [ ]:
### Change it to NDCG


In [ ]:
metric=None
if args.value_metric=="DCG":
    metric=Metrics.DCG(data, args.length_ranking)
elif args.value_metric=="NDCG":
    metric=Metrics.NDCG(data, args.length_ranking, args.replacement)
elif args.value_metric=="ERR":
    metric=Metrics.ERR(data, args.length_ranking)
elif args.value_metric=="MaxRel":
    metric=Metrics.MaxRelevance(data, args.length_ranking)
elif args.value_metric=="SumRel":
    metric=Metrics.SumRelevance(data, args.length_ranking)
else:
    print("Parallel:main [ERR] Metric %s not supported." % args.value_metric, flush=True)
    sys.exit(0)

In [ ]:
estimator

In [ ]:
### IPS LOG
numQueries=len(data.docsPerQuery)
trueMetric=numpy.zeros(numQueries, dtype=numpy.float64)
for i in range(numQueries):
    trueMetric[i]=metric.computeMetric(i, targetPolicy.predict(i, args.length_ranking))
    if i%100==0:
        print(".", end="", flush=True)
print("", flush=True)

target=trueMetric.mean(dtype=numpy.float64)
print("Parallel:main [LOG] *** TARGET: ", target, flush = True)
del trueMetric

In [ ]:
### DM LOG
numQueries=len(data.docsPerQuery)
trueMetric=numpy.zeros(numQueries, dtype=numpy.float64)
for i in range(numQueries):
    trueMetric[i]=metric.computeMetric(i, targetPolicy.predict(i, args.length_ranking))
    if i%100==0:
        print(".", end="", flush=True)
print("", flush=True)

target=trueMetric.mean(dtype=numpy.float64)
print("Parallel:main [LOG] *** TARGET: ", target, flush = True)
del trueMetric

In [ ]:
saveValues = numpy.linspace(start=int(args.logSize/args.saveSize), stop=args.logSize, num=args.saveSize, endpoint=True, dtype=numpy.int)
saveValues

In [ ]:
outputString = args.output_dir+'ssynth_'+args.value_metric+'_'+args.dataset+'_'
if args.max_docs is None:
    outputString += '-1_'
else:
    outputString += str(args.max_docs)+'_'

outputString += str(args.length_ranking) +'_'
if args.replacement:
    outputString += 'r'
else:
    outputString += 'n'
outputString += str(float(args.temperature)) + '_' 
outputString += 'f' + args.logging_ranker + '_e' + args.evaluation_ranker + '_' + str(args.numpy_seed)
outputString += '_'+args.approach
if args.approach.startswith("DM"):
    outputString += '_'+str(args.trainingSize)

In [101]:
for iteration in range(args.start, args.stop):
    iterOutputString = outputString+'_'+str(iteration)+'.z'
    if os.path.isfile(iterOutputString):
        print("Parallel:main [LOG] *** Found %s, skipping" % iterOutputString, flush=True)
        continue

    # Reset estimator
    estimator.reset()

    # reset output
    saveMSEs = numpy.zeros(args.saveSize, numpy.float64)
    savePreds = numpy.zeros(args.saveSize, numpy.float64)

    numpy.random.seed(args.numpy_seed + 7*iteration)
    currentSaveIndex=0
    currentSaveValue=saveValues[currentSaveIndex]-1

    loggedData=None
    if args.trainingSize > 0:
        loggedData=[]

    for j in range(args.logSize):
        currentQuery=numpy.random.randint(0, numQueries)
        loggedRanking=loggingPolicy.predict(currentQuery, args.length_ranking)
        loggedValue=metric.computeMetric(currentQuery, loggedRanking)

        newRanking=targetPolicy.predict(currentQuery,args.length_ranking)

        estimatedValue=None
        if (args.trainingSize > 0 and j < args.trainingSize):
            estimatedValue=0.0
            loggedData.append((currentQuery, loggedRanking, loggedValue))
        else:
            if j==args.trainingSize:
                try:
                    estimator.train(loggedData)
                    if args.approach.startswith("DMc"):
                        estimator.estimateAll(metric=metric)
                    else:
                        estimator.estimateAll()
                except AttributeError:
                    pass

            estimatedValue=estimator.estimate(currentQuery, loggedRanking, newRanking, loggedValue)

        if j==currentSaveValue:
            savePreds[currentSaveIndex]=estimatedValue
            saveMSEs[currentSaveIndex]=(estimatedValue-target)**2
            currentSaveIndex+=1
            if currentSaveIndex<args.saveSize:
                currentSaveValue=saveValues[currentSaveIndex]-1

        if j%1000==0:
            print(".", end = "", flush = True)
            numpy.random.seed(args.numpy_seed + 7*iteration + j + 1)

    print("")
    print("Parallel:main [LOG] Iter:%d Truth Estimate=%0.5f" % (iteration, target), flush = True)
    print("Parallel:main [LOG] %s Estimate=%0.5f MSE=%0.3e" % (args.approach, savePreds[-1], saveMSEs[-1]), flush=True)

    ##joblib.dump((saveValues, saveMSEs, savePreds, target), iterOutputString)




..............................
Parallel:main [LOG] Iter:1 Truth Estimate=0.18710
Parallel:main [LOG] IPS Estimate=243.02253 MSE=5.897e+04
..............................
Parallel:main [LOG] Iter:2 Truth Estimate=0.18710
Parallel:main [LOG] IPS Estimate=202.34533 MSE=4.087e+04
..............................
Parallel:main [LOG] Iter:3 Truth Estimate=0.18710
Parallel:main [LOG] IPS Estimate=219.37959 MSE=4.805e+04
..............................
Parallel:main [LOG] Iter:4 Truth Estimate=0.18710
Parallel:main [LOG] IPS Estimate=330.72815 MSE=1.093e+05
..............................
Parallel:main [LOG] Iter:5 Truth Estimate=0.18710
Parallel:main [LOG] IPS Estimate=303.82821 MSE=9.220e+04
..............................
Parallel:main [LOG] Iter:6 Truth Estimate=0.18710
Parallel:main [LOG] IPS Estimate=512.39066 MSE=2.624e+05
..............................
Parallel:main [LOG] Iter:7 Truth Estimate=0.18710
Parallel:main [LOG] IPS Estimate=321.73316 MSE=1.034e+05
..............................
Par

In [ ]:
### change target policy

In [ ]:
numpy.random.seed(387)
detLogger=DeterministicPolicy(data, 'ridge')
detLogger.train(anchorURLFeatures, 'url')

detLogger.filterDataset(100)
data=detLogger.dataset
del detLogger

In [ ]:
#Setup target policy
numpy.random.seed(args.numpy_seed)
targetPolicy=DeterministicPolicy(data, 'ridge')
targetPolicy.train(bodyTitleDocFeatures, 'body')
targetPolicy.predictAll(args.length_ranking)

loggingPolicy=None
if args.temperature <= 0.0:
    loggingPolicy=UniformPolicy(data, args.replacement)

else:
    underlyingPolicy=DeterministicPolicy(data, args.logging_ranker)
    underlyingPolicy.train(anchorURLFeatures, 'url')
    loggingPolicy=NonUniformPolicy(underlyingPolicy, data, args.replacement, args.temperature)

loggingPolicy.setupGamma(args.length_ranking)

smallestProb=1.0
docSet=set(data.docsPerQuery)
for i in docSet:
    currentMin=None
    if args.temperature > 0.0:
        currentMin=numpy.amin(loggingPolicy.multinomials[i])
    else:
        currentMin=1.0/i
    if currentMin < smallestProb:
        smallestProb=currentMin
print("Parallel:main [LOG] Temperature:", args.temperature, "\t Smallest marginal probability:", smallestProb, flush=True)

In [ ]:
### Change it to random forest
import Estimators
import imp
imp.reload(Estimators)
estimator=None
if args.approach=="OnPolicy":
    estimator=Estimators.OnPolicy(args.length_ranking, loggingPolicy, targetPolicy, metric)
    estimator.estimateAll()
elif args.approach=="IPS":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformIPS(args.length_ranking, loggingPolicy, targetPolicy)
    else:    
        estimator=Estimators.UniformIPS(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach=="IPS_SN":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformSNIPS(args.length_ranking, loggingPolicy, targetPolicy)
    else:
        estimator=Estimators.UniformSNIPS(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach=="PI":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformPI(args.length_ranking, loggingPolicy, targetPolicy)
    else:
        estimator=Estimators.UniformPI(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach=="PI_SN":
    if args.temperature > 0.0:
        estimator=Estimators.NonUniformSNPI(args.length_ranking, loggingPolicy, targetPolicy)
    else:
        estimator=Estimators.UniformSNPI(args.length_ranking, loggingPolicy, targetPolicy)
elif args.approach.startswith("DM"):
    estimatorType='rf'
    estimator=Estimators.Direct(args.length_ranking, loggingPolicy, targetPolicy, estimatorType)
else:
    print("Parallel:main [ERR] Estimator %s not supported." % args.approach, flush=True)
    sys.exit(0)

In [ ]:
metric=None
if args.value_metric=="DCG":
    metric=Metrics.DCG(data, args.length_ranking)
elif args.value_metric=="NDCG":
    metric=Metrics.NDCG(data, args.length_ranking, args.replacement)
elif args.value_metric=="ERR":
    metric=Metrics.ERR(data, args.length_ranking)
elif args.value_metric=="MaxRel":
    metric=Metrics.MaxRelevance(data, args.length_ranking)
elif args.value_metric=="SumRel":
    metric=Metrics.SumRelevance(data, args.length_ranking)
else:
    print("Parallel:main [ERR] Metric %s not supported." % args.value_metric, flush=True)
    sys.exit(0)

In [ ]:
### IPS LOG
numQueries=len(data.docsPerQuery)
trueMetric=numpy.zeros(numQueries, dtype=numpy.float64)
for i in range(numQueries):
    trueMetric[i]=metric.computeMetric(i, targetPolicy.predict(i, args.length_ranking))
    if i%100==0:
        print(".", end="", flush=True)
print("", flush=True)

target=trueMetric.mean(dtype=numpy.float64)
print("Parallel:main [LOG] *** TARGET: ", target, flush = True)
del trueMetric

In [ ]:
### DM LOG
numQueries=len(data.docsPerQuery)
trueMetric=numpy.zeros(numQueries, dtype=numpy.float64)
for i in range(numQueries):
    trueMetric[i]=metric.computeMetric(i, targetPolicy.predict(i, args.length_ranking))
    if i%100==0:
        print(".", end="", flush=True)
print("", flush=True)

target=trueMetric.mean(dtype=numpy.float64)
print("Parallel:main [LOG] *** TARGET: ", target, flush = True)
del trueMetric

In [ ]:
saveValues = numpy.linspace(start=int(args.logSize/args.saveSize), stop=args.logSize, num=args.saveSize, endpoint=True, dtype=numpy.int)
saveValues

In [ ]:
outputString = args.output_dir+'ssynth_'+args.value_metric+'_'+args.dataset+'_'+targetPolicy.name
if args.max_docs is None:
    outputString += '-1_'
else:
    outputString += str(args.max_docs)+'_'

outputString += str(args.length_ranking) +'_'
if args.replacement:
    outputString += 'r'
else:
    outputString += 'n'
outputString += str(float(args.temperature)) + '_' 
outputString += 'f' + args.logging_ranker + '_e' + args.evaluation_ranker + '_' + str(args.numpy_seed)
outputString += '_'+args.approach
if args.approach.startswith("DM"):
    outputString += '_'+str(args.trainingSize)

In [ ]:
for iteration in range(args.start, args.stop):
    iterOutputString = outputString+'_'+str(iteration)+'.z'
    if os.path.isfile(iterOutputString):
        print("Parallel:main [LOG] *** Found %s, skipping" % iterOutputString, flush=True)
        continue

    # Reset estimator
    estimator.reset()

    # reset output
    saveMSEs = numpy.zeros(args.saveSize, numpy.float64)
    savePreds = numpy.zeros(args.saveSize, numpy.float64)

    numpy.random.seed(args.numpy_seed + 7*iteration)
    currentSaveIndex=0
    currentSaveValue=saveValues[currentSaveIndex]-1

    loggedData=None
    if args.trainingSize > 0:
        loggedData=[]

    for j in range(args.logSize):
        currentQuery=numpy.random.randint(0, numQueries)
        loggedRanking=loggingPolicy.predict(currentQuery, args.length_ranking)
        loggedValue=metric.computeMetric(currentQuery, loggedRanking)

        newRanking=targetPolicy.predict(currentQuery,args.length_ranking)

        estimatedValue=None
        if (args.trainingSize > 0 and j < args.trainingSize):
            estimatedValue=0.0
            loggedData.append((currentQuery, loggedRanking, loggedValue))
        else:
            if j==args.trainingSize:
                try:
                    estimator.train(loggedData)
                    if args.approach.startswith("DMc"):
                        estimator.estimateAll(metric=metric)
                    else:
                        estimator.estimateAll()
                except AttributeError:
                    pass

            estimatedValue=estimator.estimate(currentQuery, loggedRanking, newRanking, loggedValue)

        if j==currentSaveValue:
            savePreds[currentSaveIndex]=estimatedValue
            saveMSEs[currentSaveIndex]=(estimatedValue-target)**2
            currentSaveIndex+=1
            if currentSaveIndex<args.saveSize:
                currentSaveValue=saveValues[currentSaveIndex]-1

        if j%1000==0:
            print(".", end = "", flush = True)
            numpy.random.seed(args.numpy_seed + 7*iteration + j + 1)

    print("")
    print("Parallel:main [LOG] Iter:%d Truth Estimate=%0.5f" % (iteration, target), flush = True)
    print("Parallel:main [LOG] %s Estimate=%0.5f MSE=%0.3e" % ('DM_rf', savePreds[-1], saveMSEs[-1]), flush=True)

    ##joblib.dump((saveValues, saveMSEs, savePreds, target), iterOutputString)



